# Notes on data so far

- Going to use this notebook to create a dataset for multiclass classification 

- Merge some stats...

There are a lot of stats that determines the player's ability. I'm going to group them like so: 

pace	

shooting	


passing	


dribbling

defending
defending_marking_awareness
defending_standing_tackle
defending_sliding_tackle!

physic



attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys

skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control

movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance

power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots

mentality_aggression
mentality_interceptions
mentality_positioning
mentality_vision
mentality_penalties
mentality_composure

Goalkeeping

height

With this grouping i will create average stats for movement, mentality, power, attacking, goalkeeper. 

**The main reason i am doing it this way is because i want simple UI for the application** 



Tasks:
 - 1 - create classification labels 
 - 2 - drop 1st set of columns
 - 3 - Fix null or nans data. 
 - 4 - Merge columns and create averages 
 - 5 - Drop 2nd set of errorneous columns
 - 6 - reorder columns
 - 7 - We will then save our engineered data. 
 - 8 - After that lets break it out train, test, valid data sets.

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import sklearn.utils
import os

In [153]:
file_path = "/Users/ryanreid/Documents/Machine Learning/fifa/archive/players_22.csv"

fifa_df = pd.read_csv(file_path, index_col=False)

/var/folders/z6/hgtgdd9502lbqgzmmg88__ym0000gn/T/ipykernel_50040/1888210308.py:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_df = pd.read_csv(file_path, index_col=False)


In [154]:
# Look at the data

fifa_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [155]:
# Look at the data
fifa_df.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,...,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,...,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,...,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,...,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,...,50.000000,29.000000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,27.000000
50%,236543.000000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1938.000000,1.000000,...,59.000000,52.000000,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,36.000000
75%,253532.500000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,111139.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,264640.000000,93.000000,95.000000,1.940000e+08,350000.000000,54.000000,206.000000,110.000000,115820.000000,5.000000,...,96.000000,93.000000,93.000000,92.000000,91.000000,92.000000,93.000000,92.000000,90.000000,65.000000


In [156]:
# Look at the data
fifa_df.shape

(19239, 110)

In [157]:
# Look at the data
fifa_df.dtypes

sofifa_id            int64
player_url          object
short_name          object
long_name           object
player_positions    object
                     ...  
player_face_url     object
club_logo_url       object
club_flag_url       object
nation_logo_url     object
nation_flag_url     object
Length: 110, dtype: object

In [158]:
# Look at the data
fifa_df.count()

sofifa_id           19239
player_url          19239
short_name          19239
long_name           19239
player_positions    19239
                    ...  
player_face_url     19239
club_logo_url       19178
club_flag_url       19178
nation_logo_url       759
nation_flag_url     19239
Length: 110, dtype: int64

In [159]:
# Look at the data
n = len(pd.unique(fifa_df['sofifa_id']))
print("number of unique vales: ", n)

number of unique vales:  19239


In [160]:
# TASK 1
# create classification labels and drop 1st set of columns

# ********************************************************************************************



In [161]:
fifa_df.club_position.unique()

array(['RW', 'ST', 'LW', 'RCM', 'GK', 'CF', 'CDM', 'LCB', 'RDM', 'RS',
       'LCM', 'SUB', 'CAM', 'RCB', 'LDM', 'LB', 'RB', 'LM', 'RM', 'LS',
       'CB', 'RES', nan, 'RWB', 'RF', 'CM', 'LWB', 'LAM', 'LF', 'RAM'],
      dtype=object)

In [162]:
fifa_df.player_positions.unique()

array(['RW, ST, CF', 'ST', 'ST, LW', 'LW, CAM', 'CM, CAM', 'GK',
       'CDM, CM', 'CF, ST', 'LM, CF, LW', 'CDM', 'CB', 'LW', 'RW',
       'CDM, RB', 'CM', 'LW, RW', 'CAM', 'RW, LW', 'CAM, RM, RW',
       'CM, LM', 'CB, CDM', 'CM, CDM', 'CF, CAM', 'LB', 'CM, CDM, CB',
       'RB', 'RM, CF, LM', 'LB, LM', 'LW, CF', 'RW, RM', 'ST, RM, RW',
       'CDM, CB', 'RB, LB', 'LM, RM, LW', 'CAM, CM, RW', 'CM, RM, ST',
       'CAM, CF, CM', 'CAM, CM', 'CAM, CF', 'ST, LW, LM', 'ST, LW, RW',
       'CF', 'CF, LW, CAM', 'RM, LM, RW', 'CM, CDM, CAM', 'LM, ST',
       'RB, RWB', 'RB, RM', 'CF, ST, CAM', 'RB, RWB, RM', 'ST, RW',
       'CB, LB', 'CAM, CM, CF', 'LW, CF, CAM', 'LW, LM, CAM', 'LM, LWB',
       'RW, CAM', 'CAM, RM, ST', 'RM, LM', 'CAM, LW, CM', 'RW, LW, CAM',
       'CB, RWB', 'CM, CAM, CDM', 'CAM, RM, LM', 'CM, CDM, LM',
       'CF, RM, RW', 'ST, CF', 'LWB, LB, LM', 'LB, LWB, LM', 'LM, CM, RM',
       'ST, RM', 'ST, CAM', 'LWB, LM', 'LB, CB', 'CAM, RM', 'RW, LW, RM',
       'RM, RW', 'CAM,

In [163]:
#  Lets strip the unique positions

# ***********************************

In [164]:
def convert_position(x):
    
    position_converter = {'RW': 'forward',
                      'ST': 'forward',
                      'LW': 'forward',
                      'CF': 'forward',
                      'RS': 'forward',
                      'RF': 'forward',
                      'LS': 'forward',
                      'LF': 'forward',
                      'RCM': 'midfielder',
                      'CDM': 'midfielder',
                      'CAM': 'midfielder',
                      'RDM': 'midfielder',
                      'LDM': 'midfielder',
                      'LM': 'midfielder',
                      'RM': 'midfielder',
                      'LAM': 'midfielder',
                      'RAM': 'midfielder',
                      'CM': 'midfielder',
                      'LCM': 'midfielder',
                      'LCB': 'defender',
                      'RCB': 'defender',
                      'LB': 'defender',
                      'RB': 'defender',
                      'CB': 'defender',
                      'RWB': 'defender',
                      'LWB': 'defender',
                      'GK': 'goalkeeper'}
    
    # Getting the first position from the player_position column     
    key = x.split(",")[0]
    
    if key not in position_converter:
        raise KeyError("value does not exist")
    else:
        return position_converter.get(key)

In [165]:
fifa_df['converted_position'] = fifa_df['player_positions'].apply(lambda x: convert_position(x))

In [166]:
fifa_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,converted_position
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,forward
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,forward
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,forward
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,forward
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,midfielder


In [167]:
def one_hot_positions(x):
    
    key = x
    
    position_converter = { 'forward': (1, 0, 0, 0),
                          'midfielder': (0, 1, 0, 0),
                          'defender': (0, 0, 1, 0),
                          'goalkeeper': (0, 0, 0, 1)
                         }
    
    if key not in position_converter:
        raise KeyError("value does not exist")
    else:
        return position_converter.get(key)

In [168]:
fifa_df['forward'], fifa_df['midfielder'], fifa_df['defender'], fifa_df['goalkeeper'] = zip(*fifa_df['converted_position'].map(one_hot_positions))

In [169]:
fifa_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,converted_position,forward,midfielder,defender,goalkeeper
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,forward,1,0,0,0
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,forward,1,0,0,0
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,forward,1,0,0,0
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,forward,1,0,0,0
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,midfielder,0,1,0,0


In [170]:
# TASK 2
#  drop 1st set of columns

# ********************************************************************************************

In [171]:
keep_columns = ['height_cm',
'pace',
'shooting',
'passing',
'dribbling',
'defending',
'physic',
'attacking_crossing',
'attacking_finishing',
'attacking_heading_accuracy',
'attacking_short_passing',
'attacking_volleys',
'skill_dribbling',
'skill_curve',
'skill_fk_accuracy',
'skill_long_passing',
'skill_ball_control',
'movement_acceleration',
'movement_sprint_speed',
'movement_agility',
'movement_reactions',
'movement_balance',
'power_shot_power',
'power_jumping',
'power_stamina',
'power_strength',
'power_long_shots',
'mentality_aggression',
'mentality_interceptions',
'mentality_positioning',
'mentality_vision',
'mentality_penalties',
'mentality_composure',
'defending_marking_awareness',
'defending_standing_tackle',
'defending_sliding_tackle',
'goalkeeping_diving',
'goalkeeping_handling',
'goalkeeping_kicking',
'goalkeeping_positioning',
'goalkeeping_reflexes',
'goalkeeping_speed',
'forward', 
'midfielder', 
'defender', 
'goalkeeper']

In [172]:
fifa_df = fifa_df[[*keep_columns]]

In [173]:
fifa_df.head()

,height_cm,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,...,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,forward,midfielder,defender,goalkeeper
0,170,85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,...,6,11,15,14,8,NaN,1,0,0,0
1,185,78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,...,15,6,12,8,10,NaN,1,0,0,0
2,187,87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,...,7,11,15,14,11,NaN,1,0,0,0
3,175,91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,...,9,9,15,15,11,NaN,1,0,0,0
4,181,76.0,86.0,93.0,88.0,64.0,78.0,94,82,55,...,15,13,5,10,13,NaN,0,1,0,0


In [174]:
# TASK 3
# Fix null or nans data.

# ********************************************************************************************



In [175]:
missing_data = fifa_df.isnull().sum()


print(missing_data[missing_data > 0])

pace                  2132
shooting              2132
passing               2132
dribbling             2132
defending             2132
physic                2132
goalkeeping_speed    17107
dtype: int64


In [176]:
# From a previous task I'm going to assign min values to the attribute columns. 
#  Reason for doing so averages would make goalkeepers better outfield players and vice versa. 

In [177]:
##-----   Code to deal with nan values  -----##

# helper function to fill NaN values with a column min
def fill_nan_with_min(df):
    '''Fills NaN values in a given dataframe with the min values in a column.
    
       :param df: a DataFrame that is assumed to contain cols of float values
       :return: returns a DataFrame with all null values replaced with the min val for a column'''
    
    # filling nan with mean value of any columns
    for col in list(df.columns.values):  
        # ignoring date/time cols
        if col != 'Date' and col != 'Time':
            df[col].iloc[:]=df[col].iloc[:].fillna(df[col].iloc[:].min())
        
    return df


# Note: going to ignore SettingWithCopyWarning for now
fifa_df = fill_nan_with_min(fifa_df)

In [178]:
fifa_df.isnull().sum()

height_cm                      0
pace                           0
shooting                       0
passing                        0
dribbling                      0
defending                      0
physic                         0
attacking_crossing             0
attacking_finishing            0
attacking_heading_accuracy     0
attacking_short_passing        0
attacking_volleys              0
skill_dribbling                0
skill_curve                    0
skill_fk_accuracy              0
skill_long_passing             0
skill_ball_control             0
movement_acceleration          0
movement_sprint_speed          0
movement_agility               0
movement_reactions             0
movement_balance               0
power_shot_power               0
power_jumping                  0
power_stamina                  0
power_strength                 0
power_long_shots               0
mentality_aggression           0
mentality_interceptions        0
mentality_positioning          0
mentality_

In [179]:
# TASK 4
#  Merge columns and create averages
# fifa_df['defending_overall'] = fifa_df[['defending','defending_marking_awareness','defending_standing_tackle','defending_sliding_tackle']].mean(axis=1)
# https://stackoverflow.com/questions/34734940/row-wise-average-for-a-subset-of-columns-with-missing-values                                                
# ********************************************************************************************

In [180]:
def multiple_column_average(columns):
    return fifa_df[columns].mean(axis=1).astype('int')

In [181]:
defending_overall = ['defending',
                     'defending_marking_awareness',
                     'defending_standing_tackle',
                     'defending_sliding_tackle']

fifa_df['defending_overall'] = multiple_column_average(defending_overall)

In [182]:
attacking_overall = ["attacking_crossing",
                     "attacking_finishing",
                     "attacking_heading_accuracy",
                     "attacking_short_passing",
                     "attacking_volleys"]

fifa_df['attacking_overall'] = multiple_column_average(attacking_overall)

In [183]:
skill_overall = ["skill_dribbling",
                 "skill_curve", 
                 "skill_fk_accuracy",
                 "skill_long_passing",
                 "skill_ball_control"]

fifa_df['skill_overall'] = multiple_column_average(skill_overall)

In [184]:
movement_overall = ["movement_acceleration",
                    "movement_sprint_speed",
                    "movement_agility",
                    "movement_reactions",
                    "movement_balance"]

fifa_df['movement_overall'] = multiple_column_average(movement_overall)

In [185]:
power_overall = ["power_shot_power",
                 "power_jumping",
                 "power_stamina",
                 "power_strength",
                 "power_long_shots"]

fifa_df['power_overall'] = multiple_column_average(power_overall)

In [186]:
mentality_overall = ["mentality_aggression",
                     "mentality_interceptions",
                     "mentality_positioning",
                     "mentality_vision",
                     "mentality_penalties",
                     "mentality_composure"]

fifa_df['mentality_overall'] = multiple_column_average(mentality_overall)

In [187]:
goalkeeping_overall = ["goalkeeping_diving",
                       "goalkeeping_handling",
                       "goalkeeping_kicking",
                       "goalkeeping_positioning",
                       "goalkeeping_reflexes",
                       "goalkeeping_speed"]

fifa_df['goalkeeping_overall'] = multiple_column_average(goalkeeping_overall)

In [188]:
fifa_df.head()

,height_cm,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,...,midfielder,defender,goalkeeper,defending_overall,attacking_overall,skill_overall,movement_overall,power_overall,mentality_overall,goalkeeping_overall
0,170,85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,...,0,0,0,28,85,94,90,77,73,11
1,185,78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,...,0,0,0,35,86,81,81,84,80,11
2,187,87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,...,0,0,0,28,87,83,85,87,74,12
3,175,91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,...,0,0,0,33,80,89,90,71,77,12
4,181,76.0,86.0,93.0,88.0,64.0,78.0,94,82,55,...,1,0,0,62,81,88,80,81,82,11


In [ ]:
# TASK 5
# Drop 2nd set of errorneous columns                                         
# ********************************************************************************************

In [189]:
keep_columns = ['height_cm',
'pace',
'shooting',
'passing',
'dribbling',
'defending_overall',
'physic',
'forward', 
'midfielder', 
'defender', 
'goalkeeper',
'attacking_overall',
'skill_overall',
'movement_overall',
'power_overall',
'mentality_overall',
'goalkeeping_overall']
    

In [190]:
fifa_df = fifa_df[[*keep_columns]]

In [191]:
fifa_df.head()

,height_cm,pace,shooting,passing,dribbling,defending_overall,physic,forward,midfielder,defender,goalkeeper,attacking_overall,skill_overall,movement_overall,power_overall,mentality_overall,goalkeeping_overall
0,170,85.0,92.0,91.0,95.0,28,65.0,1,0,0,0,85,94,90,77,73,11
1,185,78.0,92.0,79.0,86.0,35,82.0,1,0,0,0,86,81,81,84,80,11
2,187,87.0,94.0,80.0,88.0,28,75.0,1,0,0,0,87,83,85,87,74,12
3,175,91.0,83.0,86.0,94.0,33,63.0,1,0,0,0,80,89,90,71,77,12
4,181,76.0,86.0,93.0,88.0,62,78.0,0,1,0,0,81,88,80,81,82,11


In [151]:
# TASK 6
# reorder columns                                      
# https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns


# ********************************************************************************************

In [192]:
cols = list(fifa_df.columns.values)

In [193]:
print(cols)

['height_cm', 'pace', 'shooting', 'passing', 'dribbling', 'defending_overall', 'physic', 'forward', 'midfielder', 'defender', 'goalkeeper', 'attacking_overall', 'skill_overall', 'movement_overall', 'power_overall', 'mentality_overall', 'goalkeeping_overall']


In [194]:
column_reorder = ['height_cm', 
 'attacking_overall', 
 'defending_overall', 
 'pace', 
 'shooting', 
 'passing', 
 'dribbling', 
 'physic',  
 'skill_overall', 
 'movement_overall', 
 'power_overall', 
 'mentality_overall', 
 'goalkeeping_overall', 
 'forward', 
 'midfielder', 
 'defender', 
 'goalkeeper']

In [195]:
fifa_df = fifa_df[[*column_reorder]]

In [196]:
fifa_df.head(10)

,height_cm,attacking_overall,defending_overall,pace,shooting,passing,dribbling,physic,skill_overall,movement_overall,power_overall,mentality_overall,goalkeeping_overall,forward,midfielder,defender,goalkeeper
0,170,85,28,85.0,92.0,91.0,95.0,65.0,94,90,77,73,11,1,0,0,0
1,185,86,35,78.0,92.0,79.0,86.0,82.0,81,81,84,80,11,1,0,0,0
2,187,87,28,87.0,94.0,80.0,88.0,75.0,83,85,87,74,12,1,0,0,0
3,175,80,33,91.0,83.0,86.0,94.0,63.0,89,90,71,77,12,1,0,0,0
4,181,81,62,76.0,86.0,93.0,88.0,78.0,88,80,81,82,11,0,1,0,0
5,188,19,17,28.0,18.0,25.0,27.0,29.0,21,61,53,34,81,0,0,0,1
6,182,82,32,97.0,88.0,80.0,92.0,77.0,80,92,82,73,9,1,0,0,0
7,193,24,13,28.0,18.0,25.0,27.0,29.0,33,57,56,43,83,0,0,0,1
8,187,23,15,28.0,18.0,25.0,27.0,29.0,28,51,53,40,80,0,0,0,1
9,188,86,42,70.0,91.0,83.0,83.0,83.0,80,74,84,81,11,1,0,0,0


In [202]:
# TASK 7
# We will then save our engineered data.                                    
# ********************************************************************************************

In [197]:
# Convert single column to int dtype.
fifa_df['pace'] = fifa_df['pace'].astype('int')
fifa_df['shooting'] = fifa_df['shooting'].astype('int')
fifa_df['passing'] = fifa_df['passing'].astype('int')
fifa_df['dribbling'] = fifa_df['dribbling'].astype('int')
fifa_df['physic'] = fifa_df['physic'].astype('int')


In [198]:
fifa_df.head()

,height_cm,attacking_overall,defending_overall,pace,shooting,passing,dribbling,physic,skill_overall,movement_overall,power_overall,mentality_overall,goalkeeping_overall,forward,midfielder,defender,goalkeeper
0,170,85,28,85,92,91,95,65,94,90,77,73,11,1,0,0,0
1,185,86,35,78,92,79,86,82,81,81,84,80,11,1,0,0,0
2,187,87,28,87,94,80,88,75,83,85,87,74,12,1,0,0,0
3,175,80,33,91,83,86,94,63,89,90,71,77,12,1,0,0,0
4,181,81,62,76,86,93,88,78,88,80,81,82,11,0,1,0,0


In [201]:
# Task 3 - We also need randomize it - it is currently sorted in overall value (descending order)
# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
# https://stackoverflow.com/questions/15772009/shuffling-permutating-a-dataframe-in-pandas
test_df = sklearn.utils.shuffle(fifa_df)


In [202]:
test_df.head(100)

,height_cm,attacking_overall,defending_overall,pace,shooting,passing,dribbling,physic,skill_overall,movement_overall,power_overall,mentality_overall,goalkeeping_overall,forward,midfielder,defender,goalkeeper
16468,172,50,47,67,49,61,65,43,61,66,48,50,11,0,1,0,0
3632,189,45,72,68,39,45,61,74,46,64,61,52,11,0,0,1,0
9203,188,43,64,60,30,47,54,74,44,58,58,50,12,0,0,1,0
225,179,76,24,89,83,74,86,67,76,86,74,66,10,1,0,0,0
1874,174,60,65,86,55,71,78,63,68,82,59,67,9,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9863,183,12,15,28,18,25,27,29,10,46,41,31,57,0,0,0,1
9201,180,57,56,68,63,65,70,62,63,71,61,63,10,0,1,0,0
1602,177,65,63,85,65,72,81,75,71,85,74,69,12,0,1,0,0
11077,185,49,63,47,44,52,49,70,43,51,64,53,11,0,0,1,0


In [203]:
# We will then save our engineered data. 

directory = 'multiclass_classification_data'

if not os.path.isdir(directory):
    os.makedirs(directory)
    
test_df.to_csv(directory + '/outfield-players.csv', index=False)

In [204]:
# https://www.sharpsightlabs.com/blog/pandas-iloc/

def split70_20_10(df):
    train = pd.DataFrame()
    valid = pd.DataFrame()
    test = pd.DataFrame()
    
    length = df.shape[0]
    
    cutoff_idx = 70 * length // 100
    train_idx = 10 * length // 100 
    
    temp_train = df.iloc[:cutoff_idx]
    temp_validation = df.iloc[cutoff_idx: cutoff_idx+train_idx]
    temp_test = df.iloc[cutoff_idx+train_idx:]
    
    train = train.append(temp_train)
    valid = valid.append(temp_validation)
    test = test.append(temp_test)
    
    return train, valid, test
    
    

In [205]:
train_outfield_df, valid_outfield_df, test_outfield_df = split70_20_10(test_df)

/var/folders/z6/hgtgdd9502lbqgzmmg88__ym0000gn/T/ipykernel_50040/2521575833.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(temp_train)
/var/folders/z6/hgtgdd9502lbqgzmmg88__ym0000gn/T/ipykernel_50040/2521575833.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  valid = valid.append(temp_validation)
/var/folders/z6/hgtgdd9502lbqgzmmg88__ym0000gn/T/ipykernel_50040/2521575833.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(temp_test)


In [207]:
# # Check that the train/test sizes are about 80/20

print('Train length: ', len(train_outfield_df))
print('Test length: ', len(valid_outfield_df))
print('Test length: ', len(test_outfield_df))
print()
print('Decimal % test data of total data: ', len(test_outfield_df)
      /(len(train_outfield_df)+len(valid_outfield_df)+len(test_outfield_df)))

Train length:  13467
Test length:  1923
Test length:  3849

Decimal % test data of total data:  0.2000623733042258


In [208]:
directory = 'multiclass_classification_data'

if not os.path.isdir(directory):
    os.makedirs(directory)
                
train_outfield_df.to_csv(directory + '/training_outfield.csv', index=False)
valid_outfield_df.to_csv(directory + '/valid_outfield.csv', index=False)
test_outfield_df.to_csv(directory + '/test_outfield.csv', index=False)